In [65]:
import pandas as pd
import numpy as np
from sklearn import model_selection, ensemble, pipeline
import sklearn.feature_selection as feature_selection
df = pd.read_csv('feature selection dataset.csv')
df.shape

(3964, 44)

In [7]:
#Split data into features and labels
features = df.columns.drop('shares')
features

Index(['kw_max_avg', 'kw_avg_avg', 'LDA_03', 'LDA_01', 'num_keywords',
       'kw_max_min', 'max_positive_polarity', 'kw_min_avg',
       'weekday_is_tuesday', 'kw_avg_min', 'abs_title_sentiment_polarity',
       'global_rate_negative_words', 'data_channel_is_entertainment',
       'rate_negative_words', 'weekday_is_monday', 'self_reference_min_shares',
       'self_reference_avg_sharess', 'n_non_stop_unique_tokens',
       'weekday_is_wednesday', 'num_imgs', 'kw_avg_max',
       'global_rate_positive_words', 'min_positive_polarity', 'kw_max_max',
       'num_videos', 'weekday_is_sunday', 'num_hrefs',
       'data_channel_is_socmed', 'rate_positive_words', 'kw_min_min',
       'data_channel_is_bus', 'max_negative_polarity', 'kw_min_max',
       'avg_negative_polarity', 'global_sentiment_polarity',
       'data_channel_is_lifestyle', 'title_sentiment_polarity',
       'data_channel_is_world', 'abs_title_subjectivity',
       'self_reference_max_shares', 'is_weekend', 'LDA_02', 'LDA_04']

In [13]:
df['shares']

0       2400
1       1300
2       7100
3       3300
4        904
        ... 
3959     847
3960    1500
3961     728
3962    4300
3963    6500
Name: shares, Length: 3964, dtype: int64

In [67]:
#Select useful features with RandomForestClassifier
selector = feature_selection.SelectFromModel(ensemble.RandomForestClassifier(min_samples_leaf = 10), threshold = 'median')
selector.fit(df[features], df['shares'])
selector.get_support()

array([ True,  True,  True,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True,  True,  True,
       False, False,  True,  True, False, False, False, False,  True,
       False,  True, False, False, False,  True,  True,  True, False,
       False, False, False,  True, False,  True,  True])

In [43]:
#Pipeline that selects feature and runs model
pipe = pipeline.Pipeline ([
    ('fs', selector),
    ('model', ensemble.RandomForestRegressor(min_samples_leaf=10))
])
pipe

Pipeline(steps=[('fs',
                 SelectFromModel(estimator=RandomForestClassifier(min_samples_leaf=10),
                                 threshold='median')),
                ('model', RandomForestRegressor(min_samples_leaf=10))])

In [47]:
#set up cross validation
xval = model_selection.KFold(10, shuffle = True)
xval

KFold(n_splits=10, random_state=None, shuffle=True)

In [51]:
#Run pipeline
result = model_selection.cross_validate(pipe, df[features], df['shares'], 
                                         cv = xval, scoring='r2', return_train_score=True,return_estimator=True)
result

{'fit_time': array([16.4746511 , 16.1117909 , 14.17402506, 14.01754427, 14.24632692,
        13.92353082, 17.29632306, 14.23164892, 13.88473868, 14.38646722]),
 'score_time': array([0.03516579, 0.03366089, 0.03449488, 0.02244186, 0.02447915,
        0.02328396, 0.02559471, 0.02305698, 0.02457213, 0.02224398]),
 'estimator': [Pipeline(steps=[('fs',
                   SelectFromModel(estimator=RandomForestClassifier(min_samples_leaf=10),
                                   threshold='median')),
                  ('model', RandomForestRegressor(min_samples_leaf=10))]),
  Pipeline(steps=[('fs',
                   SelectFromModel(estimator=RandomForestClassifier(min_samples_leaf=10),
                                   threshold='median')),
                  ('model', RandomForestRegressor(min_samples_leaf=10))]),
  Pipeline(steps=[('fs',
                   SelectFromModel(estimator=RandomForestClassifier(min_samples_leaf=10),
                                   threshold='median')),
         

In [61]:
#Calculate accuracy score
train_r2 = np.mean(result['train_score'])
train_r2_sd = np.std(result['train_score'])
test_r2 = np.mean(result['test_score'])
test_r2_sd = np.std(result['test_score'])
train_r2

np.float64(0.26224890809899215)